# Test Helmholtz Model


In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
import sys
import torch
import gc
import numpy as np

Import local packages

In [11]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../.."))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.utils.logger import Logging
from src.data.helmholtz_dataset import generate_training_dataset, u, f
from src.nn.pde import helmholtz_operator

Set various constant variables: model path, name, etc. We choose the best performing models from each architecture.

In [16]:
TANH_NORM_DIST = "../../model/a1/helmholtz_tanh_normal_a1.pth"
TANH_PARAM_DIST = "../../model/a2/helmholtz_tanh_trainable_a2.pth"
BSPLINE_DIST = "../../model/a1/helmholtz_bspline_a1.pth"
GRBF_DIST = "../../model/a2/helmholtz_grbf_a2.pth"
FOURIER_DIST = "../../model/a1/helmholtz_fourier_a1.pth"
CHEBTSHEV_DIST = "../../model/a1/helmholtz_chebyshev_a1.pth"
JACOBI_DIST = "../../model/a1/helmholtz_jacobi_a1.pth"


MODEL_PATH_LIST = {
    "tanh": TANH_NORM_DIST,
    "param_tanh": TANH_PARAM_DIST,
    "grbf": GRBF_DIST,
    "bspline": BSPLINE_DIST,
    "chebyshev": CHEBTSHEV_DIST,
    "jacobi": JACOBI_DIST,
    "fourier": FOURIER_DIST,
}

SOLVER_TO_MODULE = {
    "KAN_GRBF": "src.nn.grbf",
    "jacobi": "src.nn.jacobi",
    "bspline": "src.nn.bspline",
    "chebyshev": "src.nn.chebyshev",
    "MLP": "src.nn.tanh_parameterized",
    "tanh": "src.nn.tanh",
    "fourier": "src.nn.fourier",
}

In [15]:
TEST_CHECKPOINT_PATH = os.path.join(PROJECT_ROOT, "result/helmholtz")
logger = Logging(TEST_CHECKPOINT_PATH)
result_dir = logger.get_output_dir()

## Generate Testing Dataset

In [7]:
A1 = 1
A2 = 4
LAMBDA = 1.0

dom_coords = torch.tensor([[-1.0, -1.0], [1.0, 1.0]], dtype=torch.float32).to("cpu")


[bcs_sampler, res_sampler] = generate_training_dataset("cpu")

# Test data
numer_of_points = 100


t = (
    torch.linspace(dom_coords[0, 0], dom_coords[1, 0], numer_of_points)
    .to("cpu")
    .unsqueeze(1)
)
x = (
    torch.linspace(dom_coords[0, 1], dom_coords[1, 1], numer_of_points)
    .to("cpu")
    .unsqueeze(1)
)
t, x = torch.meshgrid(t.squeeze(), x.squeeze())
X_star = torch.hstack((t.flatten().unsqueeze(1), x.flatten().unsqueeze(1))).to("cpu")

# Exact solution
u_star = u(X_star, A1, A2)
f_star = f(X_star, A1, A2, LAMBDA)


logger.print(f"Output directory {result_dir=}")

/home/ubuntu/miniconda3/envs/pytorch-2.3.1-corona-pinn/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905969824/work/aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
INFO:src.utils.logger:Output directory result_dir='/home/ubuntu/afrah/code/pinn_learnable_activation/result/helmholtz/2024-10-05_16-12-36-165005'


## Load the saved models, Test and Print Prediction Accuracy

In [14]:
for activation, model_path in MODEL_PATH_LIST.items():
    logger.print(f"MODEL_PATH {model_path}")
    # Load the state from the saved model
    state = torch.load(
        model_path,
    )
    config = state.get("config", {})
    solver = config.get("solver")

    # Extract model configuration from state
    model_activation_name = config.get("activation")
    model_architecture = config.get("network")
    loss_dict = state.get("loss_history")
    # Dynamically import the correct module and class
    if solver in SOLVER_TO_MODULE:
        module = __import__(SOLVER_TO_MODULE[solver], fromlist=["PINNKAN"])
        PINNKAN = getattr(module, "PINNKAN")

        # Initialize fluid and solid models
        model = PINNKAN(model_architecture, model_activation_name).to("cpu")

    model.load_state_dict(state["model_state_dict"])

    logger.print(f"activation:  , {model_activation_name}")
    logger.print(f"problem: {config.get('problem')}")
    logger.print(f"solver:  , {solver}")
    logger.print(f"network:  , {config.get('network')}")
    logger.print(f"term loss weights :  , {config.get('weights')}")

    [u_pred, f_pred] = helmholtz_operator(model, X_star[:, 0:1], X_star[:, 1:2])
    if u_pred.is_cuda:
        u_pred = u_pred.cpu()
        f_pred = f_pred.cpu()

    logger.print(
        f"Model {activation} with iterations: {len(loss_dict[next(iter(loss_dict))])}"
    )

    X_analytic = X_star.detach().numpy()
    u_analytic = u_star.detach().numpy()
    f_analytic = f_star.detach().numpy()
    u_pred = u_pred.detach().numpy()
    f_pred = f_pred.detach().numpy()

    error_u = (
        np.linalg.norm(u_analytic - u_pred, 2) / np.linalg.norm(u_analytic, 2)
    ) * 100.0
    error_f = (
        np.linalg.norm(f_analytic - f_pred, 2) / np.linalg.norm(f_analytic + 1e-9, 2)
    ) * 100.0

    logger.print("Relative L2 error_u: {:.2e}".format(error_u))
    logger.print("Relative L2 error_f: {:.2e}".format(error_f))

    for key in loss_dict:
        logger.print("Final loss %s: %e" % (key, loss_dict[key][-1]))

    logger.print("******************************\n")
    del model
    gc.collect()
    torch.cuda.empty_cache()
logger.print("file directory:", logger.get_output_dir())

INFO:src.utils.logger:MODEL_PATH ../../model/a1/helmholtz_tanh_normal_a1.pth
INFO:src.utils.logger:activation:  , None
INFO:src.utils.logger:problem: helmholtz


INFO:src.utils.logger:solver:  , tanh
INFO:src.utils.logger:network:  , [2, 30, 30, 30, 1]
INFO:src.utils.logger:term loss weights :  , [10.0, 1.0]
INFO:src.utils.logger:Model tanh with iterations: 60001
INFO:src.utils.logger:Relative L2 error_u: 6.13e+00
INFO:src.utils.logger:Relative L2 error_f: 3.09e+00
INFO:src.utils.logger:Final loss lbcs: 6.601419e-03
INFO:src.utils.logger:Final loss lphy: 4.367668e+00
INFO:src.utils.logger:******************************

INFO:src.utils.logger:MODEL_PATH ../../model/a2/helmholtz_tanh_trainable_a2.pth
INFO:src.utils.logger:activation:  , tanh
INFO:src.utils.logger:problem: None
INFO:src.utils.logger:solver:  , MLP
INFO:src.utils.logger:network:  , [2, 300, 300, 300, 1]
INFO:src.utils.logger:term loss weights :  , [10.0, 1.0]
INFO:src.utils.logger:Model param_tanh with iterations: 60001
INFO:src.utils.logger:Relative L2 error_u: 4.46e+00
INFO:src.utils.logger:Relative L2 error_f: 9.38e-01
INFO:src.utils.logger:Final loss lbcs: 3.098755e-03
INFO:src